# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Part 1 data file
cities_metadata_path = "../WeatherPy/city_data.csv"

# Read the metadata 
cities_metadata_df = pd.read_csv(cities_metadata_path)

In [3]:
cities_metadata_df.columns

Index(['City', 'Lat', 'Long', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Time'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
#Figure(layout=FigureLayo# Customise the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [34]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_metadata_df[["Lat", "Long"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
location_hum = cities_metadata_df.dropna()
loc_humidity = cities_metadata_df["Humidity"].astype(float)


In [35]:
# Create a humidity heatmap layer
fig = gmaps.figure(map_type='HYBRID', center = (0, 30) ,zoom_level = 2)

humidity_layer = gmaps.heatmap_layer(locations, weights=loc_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# Narrow down the DataFrame to find your ideal weather condition

#A max temperature lower than 30 degrees but higher than 25.
#Wind speed less than 4.5 m/s.
#Zero cloudiness.

ideal_max_temp = 32
ideal_min_temp = 24
ideal_wind_speed = 4.5
ideal_cloudiness = 0 

#Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

hotel_df = cities_metadata_df[(cities_metadata_df["Max Temp"] <= ideal_max_temp) &
                              (cities_metadata_df["Max Temp"] >= ideal_min_temp) &
                              (cities_metadata_df["Wind Speed"] <= ideal_wind_speed) &
                              (cities_metadata_df["Cloudiness"] == ideal_cloudiness)] 

hotel_df.head(20)


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time
0,Ponta do Sol,32.6667,-17.1000,25.40,74,0,1.37,PT,<module 'time' (built-in)>
22,Bengkulu,-3.8004,102.2655,24.45,84,0,1.89,ID,<module 'time' (built-in)>
34,Saint George,37.1041,-113.5841,27.66,54,0,3.09,US,<module 'time' (built-in)>
111,San Angelo,31.4638,-100.4370,30.53,50,0,3.60,US,<module 'time' (built-in)>
204,Nejapa,13.8138,-89.2306,26.78,92,0,2.06,SV,<module 'time' (built-in)>
222,Russell,32.3502,-85.1999,31.09,74,0,0.00,US,<module 'time' (built-in)>
258,Sussex,41.1834,-74.6663,30.60,73,0,2.06,US,<module 'time' (built-in)>
394,‘Ewa Beach,21.3156,-158.0072,24.22,81,0,2.57,US,<module 'time' (built-in)>
458,Vilhena,-12.7406,-60.1458,26.12,65,0,2.06,BR,<module 'time' (built-in)>
471,Dvinskoy Bereznik,62.8551,42.7072,27.48,46,0,2.68,RU,<module 'time' (built-in)>


In [37]:
hotel_df.shape

(10, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.head(20)


C:\Users\dalem\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time,Hotel Name
0,Ponta do Sol,32.6667,-17.1000,25.40,74,0,1.37,PT,<module 'time' (built-in)>,
1,Bengkulu,-3.8004,102.2655,24.45,84,0,1.89,ID,<module 'time' (built-in)>,
2,Saint George,37.1041,-113.5841,27.66,54,0,3.09,US,<module 'time' (built-in)>,
3,San Angelo,31.4638,-100.4370,30.53,50,0,3.60,US,<module 'time' (built-in)>,
4,Nejapa,13.8138,-89.2306,26.78,92,0,2.06,SV,<module 'time' (built-in)>,
5,Russell,32.3502,-85.1999,31.09,74,0,0.00,US,<module 'time' (built-in)>,
6,Sussex,41.1834,-74.6663,30.60,73,0,2.06,US,<module 'time' (built-in)>,
7,‘Ewa Beach,21.3156,-158.0072,24.22,81,0,2.57,US,<module 'time' (built-in)>,
8,Vilhena,-12.7406,-60.1458,26.12,65,0,2.06,BR,<module 'time' (built-in)>,
9,Dvinskoy Bereznik,62.8551,42.7072,27.48,46,0,2.68,RU,<module 'time' (built-in)>,


In [39]:
hotel_df.dtypes

City           object
Lat           float64
Long          float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Time           object
Hotel Name     object
dtype: object

In [40]:
hotel_df.shape

(10, 10)

In [41]:
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time,Hotel Name
0,Ponta do Sol,32.6667,-17.1000,25.40,74,0,1.37,PT,<module 'time' (built-in)>,
1,Bengkulu,-3.8004,102.2655,24.45,84,0,1.89,ID,<module 'time' (built-in)>,
2,Saint George,37.1041,-113.5841,27.66,54,0,3.09,US,<module 'time' (built-in)>,
3,San Angelo,31.4638,-100.4370,30.53,50,0,3.60,US,<module 'time' (built-in)>,
4,Nejapa,13.8138,-89.2306,26.78,92,0,2.06,SV,<module 'time' (built-in)>,
5,Russell,32.3502,-85.1999,31.09,74,0,0.00,US,<module 'time' (built-in)>,
6,Sussex,41.1834,-74.6663,30.60,73,0,2.06,US,<module 'time' (built-in)>,
7,‘Ewa Beach,21.3156,-158.0072,24.22,81,0,2.57,US,<module 'time' (built-in)>,
8,Vilhena,-12.7406,-60.1458,26.12,65,0,2.06,BR,<module 'time' (built-in)>,
9,Dvinskoy Bereznik,62.8551,42.7072,27.48,46,0,2.68,RU,<module 'time' (built-in)>,


In [42]:
# Set the hotel search radius

search_type = "lodging"
search_radius = 5000

search_loc = "" 


params = {
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get location from df
    search_city = row['City']
 
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    params["location"] = search_loc
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    search_result = requests.get(base_url, params=params)
    hotel_name = search_result.json()
    
    #Store the first hotel in dataframe
    
    try:
    # Print the name and address of the first hotel to appear
        print(f"A hotel in {search_city} to try is {hotel_name['results'][0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "N/A"
        print("No results... skipping.")
        
    print("----------------------------")

A hotel in Ponta do Sol to try is Hotel do Campo.
----------------------------
A hotel in Bengkulu to try is Grage Hotel Bengkulu.
----------------------------
A hotel in Saint George to try is Best Western Plus Abbey Inn.
----------------------------
A hotel in San Angelo to try is Motel 6 San Angelo, TX.
----------------------------
A hotel in Nejapa to try is Bolsa de Empleo MTPS- Nejapa.
----------------------------
A hotel in Russell to try is Macs.
----------------------------
A hotel in Sussex to try is Top O Hill Farm - McDougals.
----------------------------
A hotel in ‘Ewa Beach to try is Base Lodging.
----------------------------
A hotel in Vilhena to try is Hotel Mirage.
----------------------------
A hotel in Dvinskoy Bereznik to try is coastal Hotel.
----------------------------


In [43]:
hotel_df.shape

(10, 10)

In [44]:
hotel_df.head(20)

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time,Hotel Name
0,Ponta do Sol,32.6667,-17.1000,25.40,74,0,1.37,PT,<module 'time' (built-in)>,Hotel do Campo
1,Bengkulu,-3.8004,102.2655,24.45,84,0,1.89,ID,<module 'time' (built-in)>,Grage Hotel Bengkulu
2,Saint George,37.1041,-113.5841,27.66,54,0,3.09,US,<module 'time' (built-in)>,Best Western Plus Abbey Inn
3,San Angelo,31.4638,-100.4370,30.53,50,0,3.60,US,<module 'time' (built-in)>,"Motel 6 San Angelo, TX"
4,Nejapa,13.8138,-89.2306,26.78,92,0,2.06,SV,<module 'time' (built-in)>,Bolsa de Empleo MTPS- Nejapa
5,Russell,32.3502,-85.1999,31.09,74,0,0.00,US,<module 'time' (built-in)>,Macs
6,Sussex,41.1834,-74.6663,30.60,73,0,2.06,US,<module 'time' (built-in)>,Top O Hill Farm - McDougals
7,‘Ewa Beach,21.3156,-158.0072,24.22,81,0,2.57,US,<module 'time' (built-in)>,Base Lodging
8,Vilhena,-12.7406,-60.1458,26.12,65,0,2.06,BR,<module 'time' (built-in)>,Hotel Mirage
9,Dvinskoy Bereznik,62.8551,42.7072,27.48,46,0,2.68,RU,<module 'time' (built-in)>,coastal Hotel


In [59]:
#Plot markers on the heatmap.
locations = hotel_df[["Lat", "Long"]]

In [60]:
# NOTE: Do not change any of the code in this cell



# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Long"]]

In [61]:
locations

,Lat,Long
0,32.6667,-17.1000
1,-3.8004,102.2655
2,37.1041,-113.5841
3,31.4638,-100.4370
4,13.8138,-89.2306
5,32.3502,-85.1999
6,41.1834,-74.6663
7,21.3156,-158.0072
8,-12.7406,-60.1458
9,62.8551,42.7072


In [62]:
locations.dropna()

,Lat,Long
0,32.6667,-17.1000
1,-3.8004,102.2655
2,37.1041,-113.5841
3,31.4638,-100.4370
4,13.8138,-89.2306
5,32.3502,-85.1999
6,41.1834,-74.6663
7,21.3156,-158.0072
8,-12.7406,-60.1458
9,62.8551,42.7072


In [63]:
locations.shape

(10, 2)

In [64]:
# Convert Hotels to list
hotels = hotel_df["Hotel Name"].tolist()

In [66]:
# Create hotel (ideal symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 0, 0, 1)',
    stroke_color='rgba(0, 0, 0, 1)', scale=4,
    info_box_content=[f"Name: {hotels}" for hotel in hotels]
)

# Display figure
fig = gmaps.figure(center = (0, 30) ,zoom_level = 2)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# Create a combined map
fig = gmaps.figure(center = (0, 30) ,zoom_level = 2)
fig.add_layer(hotel_layer)

fig.add_layer(humidity_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))